In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import optuna

from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

###############################
# 1) Device Setup
###############################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

###############################
# 2) Category Columns
###############################
category_columns = [
    "Unlawful detention",
    "Human trafficking",
    "Enslavement",
    "Willful killing of civilians",
    "Mass execution",
    "Kidnapping",
    "Extrajudicial killing",
    "Forced disappearance",
    "Damage or destruction of civilian critical infrastructure",
    "Damage or destruction, looting, or theft of cultural heritage",
    "Military operations (battle, shelling)",
    "Gender-based or other conflict-related sexual violence",
    "Violent crackdowns on protesters/opponents/civil rights abuse",
    "Indiscriminate use of weapons",
    "Torture or indications of torture",
    "Persecution based on political, racial, ethnic, gender, or sexual orientation",
    "Movement of military, paramilitary, or other troops and equipment"
]

###############################
# 3) Load CSVs
###############################
train_df = pd.read_csv("train.csv")
val_df   = pd.read_csv("val.csv")
test_df  = pd.read_csv("test.csv")

###############################
# 4) Custom Dataset class
###############################
class ArticleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

###############################
# 5) Tokenization
###############################
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(
    list(train_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
val_encodings   = tokenizer(
    list(val_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
test_encodings  = tokenizer(
    list(test_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)

###############################
# 6) Prepare Labels & Dataset
###############################
train_labels = train_df[category_columns].values
val_labels   = val_df[category_columns].values
test_labels  = test_df[category_columns].values

train_dataset = ArticleDataset(train_encodings, train_labels)
val_dataset   = ArticleDataset(val_encodings, val_labels)
test_dataset  = ArticleDataset(test_encodings, test_labels)

###############################
# 7) compute_metrics function
###############################
def compute_metrics(p):
    # p.predictions are logits; p.label_ids are the ground truth
    preds = torch.sigmoid(torch.tensor(p.predictions))
    preds = (preds > 0.5).int().cpu().numpy()
    labels = torch.tensor(p.label_ids).cpu().numpy()

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

###############################
# 8) Optuna Objective
###############################
def objective(trial):
    # Hyperparameter search space
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    weight_decay = trial.suggest_uniform('weight_decay', 0.01, 0.1)
    num_train_epochs = trial.suggest_int('num_train_epochs', 5, 20)

    # Model setup
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=len(category_columns)
    )
    model.to(device)

    # Trainer settings
    training_args = TrainingArguments(
        output_dir='./results',
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        logging_dir='./logs',
        logging_steps=10,
        disable_tqdm=True
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    # Train & evaluate
    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_f1"]

###############################
# 9) Run Optuna Study
###############################
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

best_params = study.best_params
print("Best Hyperparameters:", best_params)

###############################
# 10) Train Final Model
###############################
final_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(category_columns)
)
final_model.to(device)

final_training_args = TrainingArguments(
    output_dir='./final_results',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_params['learning_rate'],
    per_device_train_batch_size=best_params['batch_size'],
    per_device_eval_batch_size=best_params['batch_size'],
    num_train_epochs=best_params['num_train_epochs'],
    weight_decay=best_params['weight_decay'],
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir='./final_logs'
)

final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

final_trainer.train()
print("Final model trained with best hyperparameters.")

# Evaluate on validation set (optional)
val_results = final_trainer.evaluate(val_dataset)
print("Validation Results (Final Model):", val_results)

###############################
# 11) Evaluate on Test Set
###############################
test_results = final_trainer.evaluate(test_dataset)
print("Multi-label Test Results:", test_results)

###############################
# 12) Single-Label Inference
###############################
# We'll now do a "single-category" prediction pass on the test set,
# picking exactly one category via argmax of logits,
# and computing the "exact match" accuracy if that category is among the true "1"s.

test_loader = DataLoader(
    test_dataset,
    batch_size=best_params['batch_size'],  # from the best hyperparams
    shuffle=False
)

final_model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for batch in test_loader:
        # Move to same device
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)

        outputs = final_model(**inputs)
        logits = outputs.logits  # shape: (batch_size, 17)

        # Convert to numpy
        probs = logits.detach().cpu().numpy()
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(probs)

y_true = np.array(y_true)  # shape: (num_samples, 17)
y_pred = np.array(y_pred)  # shape: (num_samples, 17)

# Single-label predictions by argmax
y_pred_single = np.argmax(y_pred, axis=1)  # shape: (num_samples,)

# "Exact-match" if predicted category is among the 1's in ground truth
accuracy = np.mean([
    1 if y_true[i, y_pred_single[i]] == 1 else 0
    for i in range(len(y_true))
])

print("Single-category exact-match accuracy on the test set:", accuracy)


c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


[I 2025-01-06 21:39:49,438] A new study created in memory with name: no-name-7ea5d9b1-5a21-449d-887e-ab58ebd586f1
C:\Users\ymasr\AppData\Local\Temp\ipykernel_19952\225130662.py:124: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
C:\Users\ymasr\AppData\Local\Temp\ipykernel_19952\225130662.py:126: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform('weight_decay', 0.01, 0.1)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model o

{'loss': 0.6389, 'grad_norm': 1.9691593647003174, 'learning_rate': 1.452098429224022e-05, 'epoch': 0.24390243902439024}
{'loss': 0.5562, 'grad_norm': 1.868700385093689, 'learning_rate': 1.399676103259256e-05, 'epoch': 0.4878048780487805}
{'loss': 0.4931, 'grad_norm': 2.98490571975708, 'learning_rate': 1.3472537772944898e-05, 'epoch': 0.7317073170731707}
{'loss': 0.4466, 'grad_norm': 0.9924401044845581, 'learning_rate': 1.2948314513297237e-05, 'epoch': 0.975609756097561}


c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to

...

{'train_runtime': 440.1007, 'train_samples_per_second': 13.211, 'train_steps_per_second': 1.677, 'train_loss': 0.20742459284257758, 'epoch': 18.0}
Final model trained with best hyperparameters.


100%|██████████| 8/8 [00:00<00:00, 12.63it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predic

Validation Results (Final Model): {'eval_loss': 0.19781076908111572, 'eval_accuracy': 0.3157894736842105, 'eval_f1': 0.5685273529243283, 'eval_precision': 0.6637060459669155, 'eval_recall': 0.5478260869565217, 'eval_runtime': 0.847, 'eval_samples_per_second': 67.295, 'eval_steps_per_second': 9.445, 'epoch': 18.0}


100%|██████████| 6/6 [00:00<00:00, 26.64it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predic

Multi-label Test Results: {'eval_loss': 0.20181123912334442, 'eval_accuracy': 0.27906976744186046, 'eval_f1': 0.515648470770422, 'eval_precision': 0.5976733143399809, 'eval_recall': 0.45555555555555555, 'eval_runtime': 0.2811, 'eval_samples_per_second': 152.997, 'eval_steps_per_second': 21.348, 'epoch': 18.0}
Single-category exact-match accuracy on the test set: 0.7441860465116279
